In [ ]:
# Import Required Libraries
import json
import pandas as pd
import os

def excel_to_assets_json(excel_file: str, json_file: str = None) -> dict:
    """
    Read an Excel file and produce a JSON file capturing the asset database
    grouped by Area -> Asset -> {land_price, house_price, rent: {...}}.

    Returns the generated assets dictionary and saves to JSON file.
    """

    # Read all sheets and take the first sheet
    excel_data = pd.read_excel(excel_file, sheet_name=None)
    df = list(excel_data.values())[0].copy()

    # Drop columns that are entirely NaN
    df = df.dropna(axis=1, how='all')

    # If the first row contains header names like 'Area' and 'Asset', use it as header
    first_row = df.iloc[0].astype(str).str.strip()
    if {'Area', 'Asset'}.issubset(set(first_row.values)):
        df.columns = first_row
        df = df.drop(df.index[0]).reset_index(drop=True)

    # Replace empty strings with NA and drop rows that are entirely NA
    df = df.replace(r'^\s*$', pd.NA, regex=True).dropna(axis=0, how='all').reset_index(drop=True)

    # Strip whitespace from object columns
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].astype(str).str.strip()

    # Convert known numeric columns to integers where possible
    numeric_cols = ['Land price', 'House price', 'Rent: 0', 'Rent: 1', 'Rent: 2', 'Rent: 3', 'Rent: 4']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

    # Select only the required columns for asset structure
    required_columns = ['Area', 'Asset', 'Land price', 'House price', 'Rent: 0', 'Rent: 1', 'Rent: 2', 'Rent: 3', 'Rent: 4']
    existing_cols = [col for col in required_columns if col in df.columns]
    df = df[existing_cols].copy()

    # Drop rows with any NaN values in the selected columns and reset index
    df = df.dropna(how='any').reset_index(drop=True)

    # Build the assets JSON from the cleaned dataframe with Area as primary key
    assets = {}
    for _, row in df.iterrows():
        area = str(row['Area']).strip()
        asset_name = str(row['Asset']).strip()
        if area not in assets:
            assets[area] = {}
        assets[area][asset_name] = {
            "land_price": int(row['Land price']),
            "house_price": int(row['House price']),
            "rent": {
                "no_houses": int(row['Rent: 0']),
                "one_house": int(row['Rent: 1']),
                "two_houses": int(row['Rent: 2']),
                "three_houses": int(row['Rent: 3']),
                "four_houses": int(row['Rent: 4'])
            }
        }

    # Determine JSON filename if not provided and write file
    if json_file is None:
        json_file = os.path.splitext(excel_file)[0] + '_assets.json'
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(assets, f, indent=2)

    return assets

In [ ]:
# Monopoly Database Structure
# Players: Player 1 and Player 2
# Assets: Each player owns different properties with house counts

database = {
    "Player 1": {
        "Boardwalk": {"houses": 4},
        "Park Place": {"houses": 3},
        "Pennsylvania Avenue": {"houses": 2}
    },
    "Player 2": {
        "Broadway": {"houses": 1},
        "Fifth Avenue": {"houses": 0},
        "Madison Avenue": {"houses": 2},
        "Wall Street": {"houses": 3}
    }
}

# Display the database as formatted JSON
print(json.dumps(database, indent=2))

In [ ]:
# Asset Structure for Anti-Monopoly Game
# Each asset has land price, house price, and variable rent based on number of houses

asset_structure = {
    "land_price": 0,           # Cost to purchase the land/property
    "house_price": 0,          # Cost to build one house on the property
    "rent": {
        "no_houses": 0,        # Rent when property has 0 houses
        "one_house": 0,        # Rent when property has 1 house
        "two_houses": 0,       # Rent when property has 2 houses
        "three_houses": 0,     # Rent when property has 3 houses
        "four_houses": 0       # Rent when property has 4 houses
    }
}

# Example: Sample assets for the Anti-Monopoly game
assets = {
    "Boardwalk": {
        "land_price": 400,
        "house_price": 200,
        "rent": {
            "no_houses": 50,
            "one_house": 100,
            "two_houses": 200,
            "three_houses": 400,
            "four_houses": 600
        }
    },
    "Park Place": {
        "land_price": 350,
        "house_price": 200,
        "rent": {
            "no_houses": 35,
            "one_house": 70,
            "two_houses": 150,
            "three_houses": 300,
            "four_houses": 500
        }
    },
    "Broadway": {
        "land_price": 220,
        "house_price": 150,
        "rent": {
            "no_houses": 18,
            "one_house": 40,
            "two_houses": 100,
            "three_houses": 240,
            "four_houses": 380
        }
    },
    "Fifth Avenue": {
        "land_price": 200,
        "house_price": 150,
        "rent": {
            "no_houses": 16,
            "one_house": 35,
            "two_houses": 80,
            "three_houses": 200,
            "four_houses": 350
        }
    }
}

# Display asset structure
print("Asset Structure Template:")
print(json.dumps(asset_structure, indent=2))
print("\n" + "="*50 + "\n")
print("Sample Assets for Anti-Monopoly Game:")
print(json.dumps(assets, indent=2))

In [9]:
excel_to_assets_json("Asset_database.xlsx")